# Unsupervised

In [28]:
# Which video is this?
vid=24
# Number of speakers?
speakers=2

In [ ]:
#for mp4 files --> .wav files
import subprocess
command = f"ffmpeg -i C:\\Users\\User\\Desktop\\internship\\data\\VOA_{vid}\\vid_full.mp4 -ab 160k -ac 1 -ar 44100 -vn data/VOA_{vid}/audio.wav"
subprocess.call(command, shell=True)

In [ ]:
# for mp3 files --> .wav files
import subprocess
command = f'ffmpeg -i C:\\Users\\User\\Desktop\\internship\\VOA_audio\\mp3\\{vid}_mp3.mp3 -ar 16000 -ac 1 -acodec pcm_s16le C:\\Users\\User\\Desktop\\internship\\VOA_audio\\wav\\{vid}_wav.wav'
subprocess.call(command, shell = True)

In [10]:
import os, sklearn.cluster
from pyAudioAnalysis.MidTermFeatures import mid_feature_extraction as mT
from pyAudioAnalysis.audioBasicIO import read_audio_file, stereo_to_mono
from pyAudioAnalysis.audioSegmentation import labels_to_segments
from pyAudioAnalysis.audioTrainTest import normalize_features
import numpy as np
import scipy.io.wavfile as wavfile
import IPython

In [30]:
# This code chunk will import the .wav file, use k-means clustering to generate segments with speaker tags, and then write 
# the segments and tags to txt file that will pop up. The format will be "start,end,tag" for each line of the txt file.

# read signal and get normalized segment feature statistics:
input_file = f"VOA_audio/wav/{vid}_wav.wav" 
fs, x = read_audio_file(input_file)
mt_size, mt_step, st_win = 2, 0.1, 0.05
[mt_feats, st_feats, _] = mT(x, fs, mt_size * fs, mt_step * fs, round(fs * st_win), round(fs * st_win * 0.5))
(mt_feats_norm, MEAN, STD) = normalize_features([mt_feats.T])
mt_feats_norm = mt_feats_norm[0].T

# perform clustering
n_clusters = speakers
x_clusters = [np.zeros((fs, )) for i in range(n_clusters)]
k_means = sklearn.cluster.KMeans(n_clusters=n_clusters)
k_means.fit(mt_feats_norm.T)
cls = k_means.labels_

# create segments and classes
segs, c = labels_to_segments(cls, mt_step)  # convert flags to segment limits

#make segs and tags ready for mach_segs\\{vid}.txt file
seg2txt = []
for i in np.arange(0,len(segs)):
    seg2txt.append(str(segs[i][0])+","+ str(segs[i][1])+","+str(c[i]))
with open(f'VOA_audio\\mach_segs\\{vid}.txt', 'w') as f:
    for line in seg2txt:
        f.write(line)
        f.write('\n')

import webbrowser
webbrowser.open(f'VOA_audio\\mach_segs\\{vid}.txt')

True

In [ ]:
import os  
#os.mkdir(f'data\\VOA_{vid}\\unsup_dir')

num_segs = np.arange(0,len(segs))
for i in num_segs:
    cur_x = x[int(segs[i, 0] * fs): int(segs[i, 1] * fs)] 
    print(f'speaker{c[i]}')
    print(f'{segs[i,0]} {segs[i,1]}')
    wavfile.write(f'data/VOA_{vid}/unsup_dir/clip{num_segs[i]}.wav', fs, np.int16(cur_x)) 
    IPython.display.display(IPython.display.Audio(f'data/VOA_{vid}/unsup_dir/clip{num_segs[i]}.wav'))

In [ ]:
import os  
#os.mkdir(f'VOA_audio\\unsup_dir')
#os.mkdir(f'VOA_audio\\unsup_dir\\{vid}')

num_segs = np.arange(0,len(segs))
for i in num_segs:
    cur_x = x[int(segs[i, 0] * fs): int(segs[i, 1] * fs)] 
    print(f'speaker{c[i]}')
    print(f'{segs[i,0]} {segs[i,1]}')
    wavfile.write(f'VOA_audio\\unsup_dir\\{vid}\\{vid}_{num_segs[i]}.wav', fs, np.int16(cur_x)) 
    IPython.display.display(IPython.display.Audio(f'VOA_audio\\unsup_dir\\{vid}\\{vid}_{num_segs[i]}.wav'))

In [ ]:
#make directory
os.mkdir(f'VOA_audio/aud_clips')
#make subdirectories
for i in np.arange(1,53):
    os.mkdir(f'VOA_audio/aud_clips/{i}')

In [8]:
for i in np.arange(1,53):
    input_file = f"VOA_audio/wav/{i}_wav.wav" 
    fs, x = read_audio_file(input_file)
    # input segments and classes
    segs_test = np.genfromtxt(f'VOA_audio/clean_segs/{i}.txt',delimiter=',')
    segs_test_s = []
    segs_test_c=[]
    for c in segs_test:
        segs_test_s.append([c[0],c[1]])
        segs_test_c.append(int(c[2]))
    segs = np.array(segs_test_s)
    cl = np.array(segs_test_c)
    
    #Write segments to audio files
    num_segs = np.arange(0,len(segs))
    for n in num_segs:
        cur_x = x[int(segs[n, 0] * fs): int(segs[n, 1] * fs)]    
        wavfile.write(f'VOA_audio/aud_clips/{i}/clip{num_segs[n]}_{cl[n]}.wav', fs, np.int16(cur_x))

In [ ]:
'''
with open(f'VOA_audio\\clean{vid}_segs.txt', 'w') as f:
    for line in seg_str:
        f.write(line)
        f.write('\n')
'''

In [ ]:
'''
from pyAudioAnalysis.audioBasicIO import read_audio_file
import numpy as np
import scipy.io.wavfile as wavfile
import IPython
import os

input_file = f"data/VOA_{vid}/audio.wav" 
fs, x = read_audio_file(input_file)
# input segments and classes
segs_test = np.genfromtxt(f'data\\VOA_{vid}\\clean_segs.txt',delimiter=',')
segs_test_s = []
segs_test_c=[]
for i in segs_test:
    segs_test_s.append([i[0],i[1]])
    segs_test_c.append(int(i[2]))
segs = np.array(segs_test_s)
cl = np.array(segs_test_c)

#make directory
os.mkdir(f'data\\VOA_{vid}\\clip_dir')

#Write segments to audio files
num_segs = np.arange(0,len(segs))
for i in num_segs:
    cur_x = x[int(segs[i, 0] * fs): int(segs[i, 1] * fs)] 
    print(f'speaker{cl[i]}')
    print(f'clip_number_{i}')
    print(f'{segs[i,0]} {segs[i,1]}')
    wavfile.write(f'data/VOA_{vid}/clip_dir/clip{num_segs[i]}_{cl[i]}.wav', fs, np.int16(cur_x))
    IPython.display.display(IPython.display.Audio(f'data/VOA_{vid}/clip_dir/clip{num_segs[i]}_{cl[i]}.wav'))
'''

# Supervised Model

In [ ]:
# Example 3: segment-level feature extraction
from pyAudioAnalysis import MidTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO 

# read audio data from file 
# (returns sampling freq and signal as a numpy array)
fs, s = aIO.read_audio_file("data/VOA_1/anna_pete_mixed.wav")

# get mid-term (segment) feature statistics 
# and respective short-term features:
mt, st, mt_n = aF.mid_feature_extraction(s, fs, 1 * fs, 1 * fs, 
                                         0.05 * fs, 0.05 * fs)
print(f'signal duration {len(s)/fs} seconds')
print(f'{st.shape[1]} {st.shape[0]}-D short-term feature vectors extracted')
print(f'{mt.shape[1]} {mt.shape[0]}-D segment feature statistic vectors extracted')
print('mid-term feature names')
for i, mi in enumerate(mt_n):
    print(f'{i}:{mi}')
    

In [ ]:
# Example6: use pyAudioAnalysis wrapper 
# to extract feature and train SVM classifier 
# for 2 speakers (2 anna_matteo/2 pete) audio samples

from pyAudioAnalysis.audioTrainTest import extract_features_and_train
mt, st = 2.25, 0.05
dirs = ["data/VOA_1/train/male_short", "data/VOA_1/train/anna_short"] 
extract_features_and_train(dirs, mt, mt, st, st, "svm_rbf", "anna_notanna")

In [ ]:
from pyAudioAnalysis import audioTrainTest as aT
files_to_test = ["data/VOA_1/train/male_short/1.wav",
                 "data/VOA_1/train/anna_short/1.wav",
                 "data/VOA_1/train/anna_short/2.wav"]
for f in files_to_test:
    print(f'{f}:')
    c, p, p_nam = aT.file_classification(f, "anna_notanna","svm_rbf")
    print(f'P({p_nam[0]}={p[0]})')
    print(f'P({p_nam[1]}={p[1]})')
    print()

In [ ]:
# Example 11
# Supervised audio segmentation example:
#  - Apply model "anna_notanna" to achieve fix-sized, supervised audio segmentation 
#    on file data/VOA_1/audio.wav
#  - Function audioSegmentation.mid_term_file_classification() uses pretrained model and applies 
#    the mid-term step that has been used when training the model (1 sec in our case as shown in Example6)
#  - data/VOA_1/lab_segs.segments contains the ground truth of the audio file

from pyAudioAnalysis.audioSegmentation import mid_term_file_classification, labels_to_segments
from pyAudioAnalysis.audioTrainTest import load_model
labels, class_names, _, _ = mid_term_file_classification("data/VOA_1/anna_pete_mixed.wav", # audio.wav 
                                                         "anna_notanna", "svm_rbf",  True, 
                                                         "data/VOA_1/lab_segs.segments")  
print("\nFix-sized segments:")
for il, l in enumerate(labels):
    print(f'fix-sized segment {il}: {class_names[int(l)]}')

# load the parameters of the model (actually we just want the mt_step here):    
cl, m, s, m_classes, mt_win, mt_step, s_win, s_step, c_beat = load_model("anna_notanna")

# print "merged" segments (use labels_to_segments())
print("\nSegments:")
segs, c = labels_to_segments(labels, mt_step)
for iS, seg in enumerate(segs):
    print(f'segment {iS} {seg[0]} sec - {seg[1]} sec: {class_names[int(c[iS])]}')